In [ ]:
import lib_tree
import ser
import strut
import copy
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, f1_score

## 2 classes

In [ ]:
Final_Data_S1= pd.read_csv("final_H358.csv")
Final_Data_S2= pd.read_csv("final_H355.csv")
Final_Data_S1.loc[Final_Data_S1[Final_Data_S1.labels.isin([ 2.0, 3.0,4.0])].index, 'labels']=1.0
Final_Data_S2.loc[Final_Data_S2[Final_Data_S2.labels.isin([ 2.0, 3.0,4.0])].index, 'labels']=1.0
ys = Final_Data_S1.labels
Xs= Final_Data_S1.drop(['Time', 'labels'], axis=1)
yt = Final_Data_S2.labels
Xt= Final_Data_S2.drop(['Time', 'labels'], axis=1)
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(Xt, yt, test_size=0.85, stratify=yt, random_state = 42)
X_train_t = X_train_t.to_numpy()
y_train_t= y_train_t.to_numpy()


In [ ]:
N_EST = 3
MAX=3
rf_or = RandomForestClassifier(n_estimators=N_EST, max_depth=MAX)
rf_or.fit(Xs, ys)
rftl= ser.SER_RF(rf_or, X_train_t, y_train_t, original_ser=True)
rf_strut = strut.STRUT_RF(rf_or, X_train_t, y_train_t)

In [ ]:
ypredMIX = np.empty(len(y_test_t))

for p in range(len(y_test_t)):
    PSER = ser.forest_posterior(rftl, np.array(X_test_t.iloc[p]).reshape(1, -1)).mean(axis=0)
    PSTRUT = strut.forest_posterior(rf_strut, np.array(X_test_t.iloc[p]).reshape(1, -1)).mean(axis=0)
    PMIX = np.array((PSER,PSTRUT))
    ypredMIX[p] = np.argmax(PMIX.mean(axis=0))


In [ ]:
mix_accuracy = accuracy_score(y_test_t, ypredMIX)
print('mix_accuracy:', mix_accuracy)

mix_accuracy: 0.8865638766519823


In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score1
precision, recall, fscore, support = score1(y_test_t, ypredMIX)
roc = roc_auc_score(y_test_t, ypredMIX)
f1_weighted = f1_score( y_test_t, ypredMIX, average='weighted')
print('weighted f1 score:', f1_weighted)
print('precision:', precision)
print('recall;', recall)
print('fscore:', fscore)
print('roc auc:', roc)
print('support:', support)
cm = confusion_matrix(y_test_t, ypredMIX)
print("Confusion Matrix:")
print(cm)

weighted f1 score: 0.880175459631862
precision: [0.88033012 0.91160221]
recall; [0.97560976 0.6547619 ]
fscore: [0.92552422 0.76212471]
roc auc: 0.8151858304297328
support: [656 252]
Confusion Matrix:
[[640  16]
 [ 87 165]]
